# Nettoyage et restructuration des données de matières premières

Ce notebook montre comment nettoyer et restructurer un DataFrame contenant des données de matières premières avec plusieurs colonnes de dates et de prix.

## Import Required Libraries
Import pandas and other necessary libraries for data manipulation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Chargement des données

In [ ]:
# Chargement du fichier CSV
df = pd.read_csv('DATA.csv')

# Affichage des premières lignes pour vérifier la structure
print("Forme du DataFrame:", df.shape)
df.head()

## 2. Analyse de la structure actuelle

In [ ]:
# Affichage des colonnes
print("Colonnes du DataFrame:")
df.columns

## 3. Restructuration des données

La stratégie sera de :
1. Créer un nouveau DataFrame propre avec une seule colonne de date
2. Ajouter chaque matière première en harmonisant les dates

In [ ]:
# Première approche : vérifier si les dates sont alignées entre les différentes matières premières
# Prenons quelques exemples pour comparer
date_cols = [col for col in df.columns if 'DATE' in col]
price_cols = [col for col in df.columns if 'DATE' not in col]

# Vérification de l'égalité des dates pour quelques colonnes
if len(df) > 0:
    print("Nombre de valeurs uniques par colonne de date:")
    for col in date_cols:
        print(f"{col}: {df[col].nunique()}")
    
    # Vérifions si les premières dates correspondent
    print("\nPremières dates de chaque colonne:")
    for col in date_cols:
        print(f"{col}: {df[col].iloc[0]}")

In [ ]:
# Méthode 1 : Si les dates sont alignées (même nombre, mêmes valeurs)
def method_aligned_dates():
    # Créer un nouveau DataFrame avec une seule colonne de date (utilisons celle du Brent comme référence)
    clean_df = pd.DataFrame()
    clean_df['Date'] = pd.to_datetime(df['DATEBrent'])
    
    # Ajouter les colonnes de prix en conservant leurs noms d'origine
    for col in price_cols:
        clean_df[col] = df[col]
    
    # Définir la date comme index
    clean_df = clean_df.set_index('Date')
    
    return clean_df

# Méthode 2 : Si les dates NE sont PAS alignées
def method_unaligned_dates():
    # Créer des DataFrames individuels pour chaque matière première
    commodities = {}
    
    # Liste des paires (date, prix) de commodités
    commodity_pairs = [
        ('DATEBrent', 'Brent'),
        ('DATECrudeOilWTI', 'CrudeOilWTI'),
        ('DATEGold', 'Gold'),
        ('DATESilver', 'Silver'),
        ('DATEHenryHub', 'HenryHub'),
        ('DATEWheat', 'Wheat'),
        ('DATESoybeans', 'Soybeans'),
        ('DATEHeatingOil', 'HeatingOil'),
        ('DATELiveCattle', 'LiveCattle'),
        ('DATEFeederCattle', 'FeederCattle'),
        ('DATECoffee', 'Coffee'),
        ('DATECotton', 'Cotton'),
        ('DATESugar', 'Sugar')
    ]
    
    # Créer un DataFrame pour chaque matière première
    for date_col, price_col in commodity_pairs:
        # Assurer que la colonne date existe
        if date_col in df.columns and price_col in df.columns:
            temp_df = df[[date_col, price_col]].copy()
            temp_df.columns = ['Date', price_col]
            temp_df['Date'] = pd.to_datetime(temp_df['Date'])
            temp_df = temp_df.dropna()
            temp_df = temp_df.set_index('Date')
            commodities[price_col] = temp_df
    
    # Fusionner tous les DataFrames sur les dates (outer join pour garder toutes les dates)
    clean_df = pd.concat(commodities.values(), axis=1)
    
    return clean_df

In [ ]:
# Choix de la méthode selon l'alignement des dates
# Pour déterminer si les dates sont alignées, vérifions si toutes les colonnes DATE ont le même nombre de valeurs uniques
if len(date_cols) > 0 and all(df[date_cols[0]].nunique() == df[col].nunique() for col in date_cols):
    print("Les dates semblent être alignées. Utilisation de la méthode 1.")
    clean_df = method_aligned_dates()
else:
    print("Les dates ne sont pas alignées. Utilisation de la méthode 2.")
    clean_df = method_unaligned_dates()

print("\nStructure du DataFrame nettoyé:")
print(f"Dimensions: {clean_df.shape}")
clean_df.head()

## 4. Vérification et traitement des valeurs manquantes

In [ ]:
# Afficher le nombre de valeurs manquantes par colonne
print("Nombre de valeurs manquantes par colonne:")
print(clean_df.isna().sum())

# Pourcentage de valeurs manquantes
print("\nPourcentage de valeurs manquantes par colonne:")
print(clean_df.isna().mean() * 100)

## 5. Options de traitement des valeurs manquantes

In [ ]:
# Option 1: Supprimer les lignes avec des valeurs manquantes (peut être trop restrictif)
clean_df_dropna = clean_df.dropna()
print(f"Dimensions après suppression des valeurs manquantes: {clean_df_dropna.shape}")

# Option 2: Remplir les valeurs manquantes avec la méthode forward fill
clean_df_ffill = clean_df.fillna(method='ffill')

# Option 3: Remplir les valeurs manquantes avec la méthode backward fill
clean_df_bfill = clean_df.fillna(method='bfill')

# Option 4: Interpolation linéaire
clean_df_interp = clean_df.interpolate(method='linear')

# Choisissons l'option d'interpolation linéaire pour cet exemple
final_df = clean_df_interp.copy()

# Vérification des valeurs manquantes restantes
print(f"Valeurs manquantes restantes: {final_df.isna().sum().sum()}")
final_df.head()

## 6. Visualisation des données

In [ ]:
# Visualiser quelques séries temporelles
plt.figure(figsize=(14, 8))

# Sélectionner quelques matières premières importantes
commodities_to_plot = ['Brent', 'Gold', 'Silver', 'Wheat']

for commodity in commodities_to_plot:
    if commodity in final_df.columns:
        # Normaliser pour permettre une comparaison sur le même graphique
        normalized = final_df[commodity] / final_df[commodity].iloc[0] * 100
        plt.plot(normalized, label=commodity)

plt.title('Évolution des prix des matières premières (normalisée, base 100)')
plt.legend()
plt.grid(True)
plt.show()

## 7. Enregistrement du DataFrame propre

In [ ]:
# Enregistrement du DataFrame nettoyé
final_df.to_csv('clean_commodities.csv')
print("DataFrame propre enregistré dans 'clean_commodities.csv'")

## 8. Résumé de la démarche

1. Nous avons chargé le fichier de données contenant les colonnes de dates et de prix pour différentes matières premières
2. Nous avons analysé la structure des données pour déterminer si les dates étaient alignées
3. Nous avons restructuré les données pour avoir un DataFrame propre avec une seule colonne de dates
4. Nous avons traité les valeurs manquantes par interpolation linéaire
5. Nous avons visualisé l'évolution des prix de quelques matières premières
6. Nous avons enregistré le DataFrame propre dans un nouveau fichier CSV

Cette approche permet d'obtenir un DataFrame propre et cohérent, prêt pour des analyses ultérieures.